In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import scipy.optimize
import pandas.io.sql as psql
import pandas as pd
import sys
sys.path.insert(0,'C:\Users\Sean Violante\Documents\Projects\lendico\lib')
import dwh
from __future__ import division

In [2]:
%matplotlib qt

In [3]:
conn = dwh.get_DWH()

In [5]:
loans_sql="""select l.*, gblrc.credit_agency_score, gblrc.pd, gblrc.pd_original, gblrc.lgd, gblrc.in_arrears_since, 
gblrc.date_of_first_loan_offer,
ranking.rating as ranking_rating, ranking.pd_start as ranking_pd_start, ranking.pd_end as ranking_pd_end,
lp.payback_day,lp.payout_date, lp.state as payback_state, lp.auto_in_arrears_since, lp.in_arrears_since in_arrears_since_man
from base.loan l
join il.global_borrower_loan_requests_cohort gblrc
on (l.dwh_country_id=gblrc.dwh_country_id and l.loan_nr=gblrc.loan_request_nr)
left join backend.ranking ranking
on (l.dwh_country_id=ranking.dwh_country_id and l.fk_ranking=ranking.id_ranking)
join base.loan_payback lp
on (l.dwh_country_id=lp.dwh_country_id and l.id_loan=lp.fk_loan)
where 
    l.state!='canceled' and 
    originated_since is not null"""

In [17]:
investor_returns_sql="""select ir.* from base.investor_return ir 
join base.loan_funding lf on (ir.dwh_country_id=lf.dwh_country_id and ir.fk_loan=lf.fk_loan and ir.fk_user=lf.fk_user)
where lf.state='funded'"""

In [7]:
loan_fundings_sql="""select * from base.loan_funding where state='funded'"""

In [8]:
%time loans=psql.read_sql(loans_sql,conn)

Wall time: 12.3 s


In [ ]:
base_returns=pd.read_csv(r'C:\Users\Sean Violante\Documents\Projects\lendico\Projects\Returns\base_rates.csv')
ds=np.array(base_returns.date.unique(),'datetime64[D]' )# do whilst treated as string


base_returns['date']=np.array(base_returns['date'],'datetime64[D]')
base_returns_stack=base_returns.set_index(['date','rating']).rename(columns=lambda x:int(x[9:])).stack()
base_returns_stack.index.names=['base_date','rating','duration_months']

base_returns_stack.name='base_return'

d1=np.array(loans.date_of_first_loan_offer,'datetime64[D]')
d2=np.searchsorted(ds,d1,side='right')
loans['base_date']=ds[d2-1]
loans['duration_months']=(loans['duration']/30).astype(np.int64)

if 'base_return' in loans.columns:
    del loans['base_return']
loans=loans.merge(base_returns_stack.reset_index(),on=['base_date','rating','duration_months'],how='left')
loans.base_return.isnull().sum()

In [9]:
%time loan_fundings=psql.read_sql(loan_fundings_sql,conn)

Wall time: 36.4 s


In [18]:
%time investor_returns=psql.read_sql(investor_returns_sql,conn)

Wall time: 17min 28s


In [10]:
loans['rating_base']=loans.rating.str[0]

In [48]:
loans.columns

Index([                 u'dwh_country_id',                         u'id_loan',
                       u'fk_loan_request',                         u'fk_user',
                            u'fk_ranking',                    u'country_name',
                         u'currency_code',      u'loan_request_creation_date',
                              u'user_age',                         u'loan_nr',
                                 u'title',                         u'purpose',
                   u'financial_situation',                     u'self_rating',
                                u'rating',                u'principal_amount',
                              u'duration',     u'nominal_interest_percentage',
             u'promo_interest_percentage',                      u'fee_amount',
                       u'interest_amount',                   u'total_payback',
                             u'is_active',                        u'category',
                    u'ranking_serialized',          

In [54]:
loan_fundings1.columns

Index([                 u'dwh_country_id',                 u'id_loan_funding',
                               u'fk_loan',                 u'fk_loan_request',
                               u'fk_user',                    u'country_name',
                         u'currency_code',      u'loan_request_creation_date',
                                u'amount',                u'investment_yield',
                                 u'state',                   u'is_authorized',
                   u'authorization_token', u'loan_request_funding_created_at',
                               u'country',              u'data_accessible_by',
                        u'investment_fee',         u'investment_return_state',
                         u'payment_count',               u'last_payment_date',
                    u'irregular_payments',                    u'close_reason',
                            u'eur_amount',                      u'created_at',
                            u'updated_at',          

In [14]:
loans['originated_since_date']=np.array(loans.payout_date,'datetime64[D]')

In [49]:
loan_fundings1=loan_fundings.merge(loans[['dwh_country_id','id_loan','originated_since_date','payback_day','principal_amount']],left_on=['dwh_country_id','fk_loan'],right_on=['dwh_country_id','id_loan'])

In [19]:
loan_fundings.head()

,dwh_country_id,id_loan_funding,fk_loan,fk_loan_request,fk_user,country_name,currency_code,loan_request_creation_date,amount,investment_yield,...,investment_return_state,payment_count,last_payment_date,irregular_payments,close_reason,eur_amount,created_at,updated_at,dwh_created,dwh_last_modified
0,1,126,8,12,829,Germany,EUR,2013-11-30,200,3.9073,...,idle,17,2015-05-26 10:44:21,None,None,200,2013-12-13 17:29:43,2015-05-26 10:44:22,2014-02-21 17:33:03.857077,2015-05-26 11:24:15.505934
1,1,135,8,12,1228,Germany,EUR,2013-11-30,100,3.8981,...,idle,17,2015-05-26 10:35:31,None,None,100,2013-12-13 17:29:43,2015-05-26 10:35:32,2014-02-21 17:33:03.857077,2015-05-26 11:24:15.505934
2,1,119,8,12,574,Germany,EUR,2013-11-30,25,3.8616,...,idle,17,2015-05-26 16:06:37,None,None,25,2013-12-13 17:29:43,2015-05-26 16:06:39,2014-02-21 17:33:03.857077,2015-05-26 16:13:46.588376
3,1,113,8,12,447,Germany,EUR,2013-11-30,25,3.8616,...,idle,17,2015-05-26 15:26:23,None,None,25,2013-12-13 17:29:42,2015-05-26 15:26:25,2014-02-21 17:33:03.857077,2015-05-26 16:13:46.588376
4,1,92,8,12,88,Germany,EUR,2013-11-30,25,3.8616,...,idle,17,2015-05-26 15:25:08,None,None,25,2013-12-13 17:29:42,2015-05-26 15:25:10,2014-02-21 17:33:03.857077,2015-05-26 16:13:46.588376


In [20]:
investor_returns.head()

,dwh_country_id,fk_loan,fk_user,interval,interval_payback_date,next_interval_payback_date,payment_amount_plan,principal_amount_plan,interest_amount_plan,initial_principal_amount_plan,...,eur_payment_amount_plan,eur_principal_amount_plan,eur_interest_amount_plan,eur_initial_principal_amount_plan,eur_sum_interval_interest_amount_plan,eur_residual_interest_amount_plan,eur_residual_principal_amount_plan,eur_payment_amount_transaction,eur_principal_amount_transaction,eur_interest_amount_transaction
0,16,1,3,6,2014-10-15,2014-11-15,112.3,108.0135,4.2865,768.8490,...,26.8259,25.8019,1.0239,183.6601,8.2736,3.0945,157.8582,26.825854,25.945768,0.880086
1,16,1,3,5,2014-09-15,2014-10-15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.825854,25.801918,1.023936
2,16,1,3,5,2014-09-15,2014-10-15,112.3,107.4147,4.8853,876.2637,...,26.8259,25.6589,1.1670,209.3190,7.2497,4.1185,183.6601,26.825854,25.658866,1.166988
3,16,1,3,4,2014-08-15,2014-09-15,112.3,106.8191,5.4809,983.0828,...,26.8259,25.5166,1.3093,234.8356,6.0827,5.2855,209.3190,26.825854,25.516606,1.309248
4,16,1,3,3,2014-07-15,2014-08-15,112.3,106.2269,6.0731,1089.3098,...,26.8259,25.3751,1.4507,260.2107,4.7734,6.5947,234.8356,NaN,NaN,NaN


In [25]:
z=investor_returns
z[(z.fk_loan==3) & (z.fk_user==38)].sort('interval_payback_date',inplace=False)

,dwh_country_id,fk_loan,fk_user,interval,interval_payback_date,next_interval_payback_date,payment_amount_plan,principal_amount_plan,interest_amount_plan,initial_principal_amount_plan,...,eur_payment_amount_plan,eur_principal_amount_plan,eur_interest_amount_plan,eur_initial_principal_amount_plan,eur_sum_interval_interest_amount_plan,eur_residual_interest_amount_plan,eur_residual_principal_amount_plan,eur_payment_amount_transaction,eur_principal_amount_transaction,eur_interest_amount_transaction
489,1,3,38,1,2014-02-15,2014-03-15,34.99,31.8545,3.1355,800.0000,...,34.99,31.8545,3.1355,800.0000,3.1355,36.6545,768.1455,34.99,31.854467,3.135533
488,1,3,38,2,2014-03-15,2014-04-15,34.99,31.9793,3.0107,768.1455,...,34.99,31.9793,3.0107,768.1455,6.1462,33.6438,736.1662,34.99,31.979318,3.010682
487,1,3,38,3,2014-04-15,2014-05-15,34.99,32.1047,2.8853,736.1662,...,34.99,32.1047,2.8853,736.1662,9.0316,30.7584,704.0616,34.99,32.104658,2.885342
486,1,3,38,4,2014-05-15,2014-06-15,34.99,32.2305,2.7595,704.0616,...,34.99,32.2305,2.7595,704.0616,11.7911,27.9989,671.8311,34.99,32.230489,2.759511
485,1,3,38,5,2014-06-15,2014-07-15,34.99,32.3568,2.6332,671.8311,...,34.99,32.3568,2.6332,671.8311,14.4243,25.3657,639.4743,34.99,32.356814,2.633186
484,1,3,38,6,2014-07-15,2014-08-15,34.99,32.4836,2.5064,639.4743,...,34.99,32.4836,2.5064,639.4743,16.9306,22.8594,606.9906,34.99,32.483634,2.506366
483,1,3,38,7,2014-08-15,2014-09-15,34.99,32.6110,2.3790,606.9906,...,34.99,32.6110,2.3790,606.9906,19.3097,20.4803,574.3797,34.99,32.610951,2.379049
482,1,3,38,8,2014-09-15,2014-10-15,34.99,32.7388,2.2512,574.3797,...,34.99,32.7388,2.2512,574.3797,21.5609,18.2291,541.6409,34.99,32.738767,2.251233
481,1,3,38,9,2014-10-15,2014-11-15,34.99,32.8671,2.1229,541.6409,...,34.99,32.8671,2.1229,541.6409,23.6838,16.1062,508.7738,34.99,32.867084,2.122916
494,1,3,38,10,2014-11-15,2014-12-15,34.99,32.9959,1.9941,508.7738,...,34.99,32.9959,1.9941,508.7738,25.6779,14.1121,475.7779,NaN,NaN,NaN


In [27]:
investor_returns.columns

Index([                       u'dwh_country_id',
                                     u'fk_loan',
                                     u'fk_user',
                                    u'interval',
                       u'interval_payback_date',
                  u'next_interval_payback_date',
                         u'payment_amount_plan',
                       u'principal_amount_plan',
                        u'interest_amount_plan',
               u'initial_principal_amount_plan',
           u'sum_interval_interest_amount_plan',
               u'residual_interest_amount_plan',
              u'residual_principal_amount_plan',
                            u'transaction_date',
                  u'payment_amount_transaction',
                u'principal_amount_transaction',
                 u'interest_amount_transaction',
                     u'eur_payment_amount_plan',
                   u'eur_principal_amount_plan',
                    u'eur_interest_amount_plan',
           u'eur_ini

In [29]:
loan_fundings.head().T

,0,1,2,3,4
dwh_country_id,1,1,1,1,1
id_loan_funding,126,135,119,113,92
fk_loan,8,8,8,8,8
fk_loan_request,12,12,12,12,12
fk_user,829,1228,574,447,88
country_name,Germany,Germany,Germany,Germany,Germany
currency_code,EUR,EUR,EUR,EUR,EUR
loan_request_creation_date,2013-11-30 00:00:00,2013-11-30 00:00:00,2013-11-30 00:00:00,2013-11-30 00:00:00,2013-11-30 00:00:00
amount,200,100,25,25,25
investment_yield,3.9073,3.8981,3.8616,3.8616,3.8616


In [70]:
def generate_cashflows_act_investor( actual,loan_funding):
    """  generate cashflows for IRR calc.
    take payments and add initial principal 
    NO HANDLING DEFAULTS
        
    """
    #if default take full outstanding borrower principal *(1-1% service fee)  * recovery fraction
       
    
    loans_orig=loan_funding[['dwh_country_id','fk_loan','fk_user','originated_since_date','eur_amount']].copy()
    loans_orig.eur_amount=-loans_orig.eur_amount
    loans_orig.rename(columns={'originated_since_date':'date' ,'eur_amount':'payment'},inplace=True)
    
    act=actual.loc[actual.transaction_date.notnull(),['dwh_country_id','fk_loan','transaction_date','fk_user','eur_payment_amount_transaction']].\
        rename(columns={'transaction_date':'date', 'eur_payment_amount_transaction':'payment'})
    #find residual at latest
    act.date=np.array(act.date,'datetime64[D]')
    
    
    cashflows=pd.concat([loans_orig, act],ignore_index=True)
    cashflows=cashflows.merge(loan_funding[['dwh_country_id', 'fk_loan','fk_user','originated_since_date']], on=['dwh_country_id','fk_loan','fk_user'])
    cashflows['dcf']=(np.datetime64('2015-04-01','D')  -cashflows["date"].values)/np.timedelta64(1,'D')/365    
    return cashflows


In [ ]:
generate_remaining_principal_act_investor(actual):
    act=actual.loc[actual.transaction_date.notnull(),['dwh_country_id','fk_loan','fk_user',\
                                                      'transaction_date','interval_payback_date',
                                                      'eur_residual_principal_amount_plan']].\
        rename(columns={'interval_payback_date':'date_outstanding', 'eur_residual_principal_amount_plan':'payment'})
    

In [34]:
loan_fundings1.columns

Index([                 u'dwh_country_id',                 u'id_loan_funding',
                               u'fk_loan',                 u'fk_loan_request',
                               u'fk_user',                    u'country_name',
                         u'currency_code',      u'loan_request_creation_date',
                                u'amount',                u'investment_yield',
                                 u'state',                   u'is_authorized',
                   u'authorization_token', u'loan_request_funding_created_at',
                               u'country',              u'data_accessible_by',
                        u'investment_fee',         u'investment_return_state',
                         u'payment_count',               u'last_payment_date',
                    u'irregular_payments',                    u'close_reason',
                            u'eur_amount',                      u'created_at',
                            u'updated_at',          

In [87]:
investor_returns1=investor_returns.sort(['interval_payback_date','transaction_date'],inplace=False)

In [119]:
z=investor_returns1
z1=z[(z.dwh_country_id==1)&(z.fk_loan==3)& (z.fk_user==38)]
z1.to_clipboard()

In [123]:
investor_returns1_gp=investor_returns1.groupby(['dwh_country_id','fk_loan','fk_user'])
def outst(x):
    """ find outstanding principal.
    for each plan date find latest paid amount and therefore corresponding remaining principal
    """
    y=x.loc[x.transaction_date.notnull(),['transaction_date','eur_residual_principal_amount_plan']]
    y['match_date']=np.array(y['transaction_date'],'datetime64[D]')-np.timedelta64(7,'D')
    
    y=y.set_index('match_date')
    idx=x.interval_payback_date.dropna().unique()
    # will get none if payment is made before plan starts
    refill=y.reindex(index=idx,method='ffill')
    return refill

outstanding_principals=investor_returns1_gp.apply(outst)

ValueError: index must be monotonic increasing or decreasing

In [124]:
%debug

> c:\anaconda\lib\site-packages\pandas\core\index.py(2403)_searchsorted_monotonic()
   2402 
-> 2403         raise ValueError('index must be monotonic increasing or decreasing')
   2404 

ipdb> u
> c:\anaconda\lib\site-packages\pandas\core\index.py(1729)_get_fill_indexer_searchsorted()
   1728         nonexact = (indexer == -1)
-> 1729         indexer[nonexact] = self._searchsorted_monotonic(target[nonexact], side)
   1730         if side == 'left':

ipdb> u
> c:\anaconda\lib\site-packages\pandas\core\index.py(1711)_get_fill_indexer()
   1710         else:
-> 1711             indexer = self._get_fill_indexer_searchsorted(target, method, limit)
   1712         return indexer

ipdb> u
> c:\anaconda\lib\site-packages\pandas\core\index.py(1697)get_indexer()
   1696         if method == 'pad' or method == 'backfill':
-> 1697             indexer = self._get_fill_indexer(target, method, limit)
   1698         elif method == 'nearest':

ipdb> u
> c:\anaconda\lib\site-packages\pandas\core\index

In [122]:
z=investor_returns1
z[(z.dwh_country_id==1)&(z.fk_loan==1479) & (z.fk_user==80357)]

,dwh_country_id,fk_loan,fk_user,interval,interval_payback_date,next_interval_payback_date,payment_amount_plan,principal_amount_plan,interest_amount_plan,initial_principal_amount_plan,...,eur_payment_amount_plan,eur_principal_amount_plan,eur_interest_amount_plan,eur_initial_principal_amount_plan,eur_sum_interval_interest_amount_plan,eur_residual_interest_amount_plan,eur_residual_principal_amount_plan,eur_payment_amount_transaction,eur_principal_amount_transaction,eur_interest_amount_transaction
1610981,1,1479,80357,0,2015-04-15,2015-05-15,68.77,0.0000,68.7700,15000.0000,...,68.77,0.0000,68.7700,15000.0000,NaN,NaN,15000.0000,NaN,NaN,NaN
1610982,1,1479,80357,1,2015-05-15,2015-06-15,456.62,381.0925,75.5275,15000.0000,...,456.62,381.0925,75.5275,15000.0000,NaN,NaN,14618.9075,456.62,381.0925,75.5275
1610983,1,1479,80357,2,2015-06-15,2015-07-15,456.62,383.0114,73.6086,14618.9075,...,456.62,383.0114,73.6086,14618.9075,NaN,NaN,14235.8961,NaN,NaN,NaN
1610984,1,1479,80357,3,2015-07-15,2015-08-15,456.62,384.9399,71.6801,14235.8961,...,456.62,384.9399,71.6801,14235.8961,NaN,NaN,13850.9562,NaN,NaN,NaN
1610985,1,1479,80357,4,2015-08-15,2015-09-15,456.62,386.8781,69.7419,13850.9562,...,456.62,386.8781,69.7419,13850.9562,NaN,NaN,13464.0781,NaN,NaN,NaN
1610986,1,1479,80357,5,2015-09-15,2015-10-15,456.62,388.8261,67.7939,13464.0781,...,456.62,388.8261,67.7939,13464.0781,NaN,NaN,13075.2520,NaN,NaN,NaN
1610987,1,1479,80357,6,2015-10-15,2015-11-15,456.62,390.7839,65.8361,13075.2520,...,456.62,390.7839,65.8361,13075.2520,NaN,NaN,12684.4681,NaN,NaN,NaN
1610988,1,1479,80357,7,2015-11-15,2015-12-15,456.62,392.7516,63.8684,12684.4681,...,456.62,392.7516,63.8684,12684.4681,NaN,NaN,12291.7165,NaN,NaN,NaN
1610989,1,1479,80357,8,2015-12-15,2016-01-15,456.62,394.7292,61.8908,12291.7165,...,456.62,394.7292,61.8908,12291.7165,NaN,NaN,11896.9873,NaN,NaN,NaN
1610990,1,1479,80357,9,2016-01-15,2016-02-15,456.62,396.7167,59.9033,11896.9873,...,456.62,396.7167,59.9033,11896.9873,NaN,NaN,11500.2706,NaN,NaN,NaN


In [128]:
z=investor_returns1.loc[investor_returns1.interval_payback_date.isnull(),['dwh_country_id','fk_loan','fk_user','transaction_date']]

In [131]:
z.to_clipboard()

In [118]:
y

transaction_date  \
dwh_country_id fk_loan fk_user match_date                    
1              3       38      2014-02-15       2014-02-18   
                               2014-03-15       2014-03-18   
                               2014-04-15       2014-04-16   
                               2014-05-15       2014-05-16   
                               2014-06-15       2014-06-17   
                               2014-07-15       2014-07-15   
                               2014-08-15       2014-08-18   
                               2014-09-15       2014-09-15   
                               2014-10-15       2014-10-15   
                               2014-11-15       2014-10-15   
                               2014-12-15       2014-12-15   
                               2015-01-15       2015-01-15   
                               2015-02-15       2015-01-15   
                               2015-03-15       2015-02-23   
                               2015-04-15       2015-04-22   
                               2015-05-15       2015-05-22   
                               2015-06-15       2015-05-22   
                               2015-07-15       2015-05-22   
                               2015-08-15       2015-05-22   
                               2015-09-15       2015-05-22   
                               2015-10-15       2015-05-22   
                               2015-11-15       2015-05-22   
                               2015-12-15       2015-05-22   
                               2016-01-15       2015-05-22   

                                           eur_residual_principal_amount_plan  
dwh_country_id fk_loan fk_user match_date                                      
1              3       38      2014-02-15                            768.1455  
                               2014-03-15                            736.1662  
                               2014-04-15                            704.0616  
                               2014-05-15                            671.8311  
                               2014-06-15                            639.4743  
                               2014-07-15                            606.9906  
                               2014-08-15                            574.3797  
                               2014-09-15                            541.6409  
                               2014-10-15                            508.7738  
                               2014-11-15                            508.7738  
                               2014-12-15                            442.6527  
                               2015-01-15                            409.3976  
                               2015-02-15                            409.3976  
                               2015-03-15                            376.0122  
                               2015-04-15                            308.8484  
                               2015-05-15                            275.0689  
                               2015-06-15                            275.0689  
                               2015-07-15                            275.0689  
                               2015-08-15                            275.0689  
                               2015-09-15                            275.0689  
                               2015-10-15                            275.0689  
                               2015-11-15                            275.0689  
                               2015-12-15                            275.0689  
                               2016-01-15                            275.0689

In [79]:
y.head(20)

transaction_date  eur_residual_payment_plan
dwh_country_id fk_loan fk_user                                               
1              3       38      0        2014-10-15                        NaN
                               1        2014-09-15                        NaN
                               2        2014-08-18                        NaN
                               3        2014-07-15                        NaN
                               4        2014-06-17                        NaN
                               5        2014-05-16                        NaN
                               6        2014-04-16                        NaN
                               7        2014-03-18                        NaN
                               8        2014-02-18                        NaN
                               9        2015-01-15                        NaN
                               10       2014-12-15                        NaN
                               11       2015-02-23                        NaN
                               12       2015-03-23                        NaN
                               13       2015-05-22                        NaN
                               14       2015-04-22                        NaN
                       59      0        2014-08-18                        NaN
                               1        2014-07-15                        NaN
                               2        2014-06-17                        NaN
                               3        2014-02-18                        NaN
                               4        2014-05-16                        NaN

In [71]:
cash=generate_cashflows_act_investor(investor_returns,loan_fundings1)

In [58]:
investor_returns.head()

,dwh_country_id,fk_loan,fk_user,interval,interval_payback_date,next_interval_payback_date,payment_amount_plan,principal_amount_plan,interest_amount_plan,initial_principal_amount_plan,...,eur_payment_amount_plan,eur_principal_amount_plan,eur_interest_amount_plan,eur_initial_principal_amount_plan,eur_sum_interval_interest_amount_plan,eur_residual_interest_amount_plan,eur_residual_principal_amount_plan,eur_payment_amount_transaction,eur_principal_amount_transaction,eur_interest_amount_transaction
0,16,1,3,6,2014-10-15,2014-11-15,112.3,108.0135,4.2865,768.8490,...,26.8259,25.8019,1.0239,183.6601,8.2736,3.0945,157.8582,26.825854,25.945768,0.880086
1,16,1,3,5,2014-09-15,2014-10-15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.825854,25.801918,1.023936
2,16,1,3,5,2014-09-15,2014-10-15,112.3,107.4147,4.8853,876.2637,...,26.8259,25.6589,1.1670,209.3190,7.2497,4.1185,183.6601,26.825854,25.658866,1.166988
3,16,1,3,4,2014-08-15,2014-09-15,112.3,106.8191,5.4809,983.0828,...,26.8259,25.5166,1.3093,234.8356,6.0827,5.2855,209.3190,26.825854,25.516606,1.309248
4,16,1,3,3,2014-07-15,2014-08-15,112.3,106.2269,6.0731,1089.3098,...,26.8259,25.3751,1.4507,260.2107,4.7734,6.5947,234.8356,NaN,NaN,NaN


In [72]:
cash[(cash.fk_loan==1) & (cash.fk_user==3)].sort('date',inplace=False)

,date,dwh_country_id,fk_loan,fk_user,payment,originated_since_date,dcf
283278,2014-04-03,16,1,3,-310.539700,2014-04-03,0.994521
283284,2014-05-15,16,1,3,26.825854,2014-04-03,0.879452
283283,2014-06-16,16,1,3,26.825854,2014-04-03,0.791781
283282,2014-08-26,16,1,3,26.825854,2014-04-03,0.597260
283281,2014-09-17,16,1,3,26.825854,2014-04-03,0.536986
283280,2014-10-14,16,1,3,26.825854,2014-04-03,0.463014
283279,2014-11-14,16,1,3,26.825854,2014-04-03,0.378082
283289,2014-12-12,16,1,3,26.825854,2014-04-03,0.301370
283288,2015-01-15,16,1,3,26.825854,2014-04-03,0.208219
283287,2015-02-13,16,1,3,26.825854,2014-04-03,0.128767


In [62]:
z=investor_returns
z[(z.fk_loan==1) & (z.fk_user==3)].sort('interval',inplace=False)

,dwh_country_id,fk_loan,fk_user,interval,interval_payback_date,next_interval_payback_date,payment_amount_plan,principal_amount_plan,interest_amount_plan,initial_principal_amount_plan,...,eur_payment_amount_plan,eur_principal_amount_plan,eur_interest_amount_plan,eur_initial_principal_amount_plan,eur_sum_interval_interest_amount_plan,eur_residual_interest_amount_plan,eur_residual_principal_amount_plan,eur_payment_amount_transaction,eur_principal_amount_transaction,eur_interest_amount_transaction
6,16,1,3,1,2014-05-15,2014-06-15,112.30,105.0523,7.2477,1300.0000,...,26.8259,25.0945,1.7313,310.5397,1.7313,9.6368,285.4452,26.825854,25.094543,1.731311
5,16,1,3,2,2014-06-15,2014-07-15,112.30,105.6380,6.6620,1194.9477,...,26.8259,25.2345,1.5914,285.4452,3.3227,8.0454,260.2107,26.825854,25.234450,1.591404
4,16,1,3,3,2014-07-15,2014-08-15,112.30,106.2269,6.0731,1089.3098,...,26.8259,25.3751,1.4507,260.2107,4.7734,6.5947,234.8356,NaN,NaN,NaN
3,16,1,3,4,2014-08-15,2014-09-15,112.30,106.8191,5.4809,983.0828,...,26.8259,25.5166,1.3093,234.8356,6.0827,5.2855,209.3190,26.825854,25.516606,1.309248
1,16,1,3,5,2014-09-15,2014-10-15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.825854,25.801918,1.023936
2,16,1,3,5,2014-09-15,2014-10-15,112.30,107.4147,4.8853,876.2637,...,26.8259,25.6589,1.1670,209.3190,7.2497,4.1185,183.6601,26.825854,25.658866,1.166988
0,16,1,3,6,2014-10-15,2014-11-15,112.30,108.0135,4.2865,768.8490,...,26.8259,25.8019,1.0239,183.6601,8.2736,3.0945,157.8582,26.825854,25.945768,0.880086
13,16,1,3,7,2014-11-15,2014-12-15,112.30,108.6157,3.6843,660.8355,...,26.8259,25.9458,0.8801,157.8582,9.1537,2.2145,131.9124,26.825854,26.090420,0.735434
12,16,1,3,8,2014-12-15,2015-01-15,112.30,109.2213,3.0787,552.2197,...,26.8259,26.0904,0.7354,131.9124,9.8891,1.4790,105.8220,NaN,NaN,NaN
10,16,1,3,9,2015-01-15,2015-02-15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.825854,26.382148,0.443706


In [ ]:
if use_outstanding:
        residual=act['fk_loan','fk_user_investor', \
                            'residual_principal_amount_investor']].\
            rename(columns={'eur_residual_principal_amount_plan':'payment'})
        
    else:
        residual=payment_plans.loc[(payment_plans.interval_payback_date>EOM_date) &\
                                   (payment_plans.fk_loan.isin(live_loans)), \
                                   ['fk_loan','fk_user_investor', 'interval_payback_date', \
                                    'initial_principal_amount_investor']]\
                                    .sort('interval_payback_date',inplace=False)\
                                    .groupby(['fk_loan','fk_user_investor']).first().reset_index()\
                                    .rename(columns={'fk_loan':'id_loan',
                                                     'initial_principal_amount_investor':'payment'})
        del residual['interval_payback_date']
    residual['date']=np.datetime64(EOM_date,'D')
    
    # for defaulted loans always use full outstanding in arrears residual['payment']=
    has_defaulted=(act_EOM.in_arrears_since_days_30360>90)
    if (has_defaulted.sum()>0):
        recovery_payment=act_EOM[has_defaulted].copy()
    
        recovery_payment['recovery']=recovery(recovery_payment['residual_principal_amount_borrower'])
        recovery_payment=recovery_payment.merge(loan_funding[['fk_loan','fk_user','loan_coverage1']],\
                                            left_on=['fk_loan','fk_user_investor'], right_on=['fk_loan','fk_user'],how='left')
        recovery_payment['payment']=.99 *recovery_payment['recovery']*recovery_payment['residual_principal_amount_borrower']\
                                    *recovery_payment['loan_coverage1']/100.0
    
    
        recovery_payment=recovery_payment[['fk_loan','fk_user_investor','payment']]\
                    .rename(columns={'fk_loan':'id_loan'})
        recovery_payment['date']=np.datetime64(EOM_date,'D')
    else:
        recovery_payment=None
    